# Comparison Of PIELM and PINNs
####
## For Formulation B
###
#### For flat plate (m=0)

In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim
import numpy as np
import matplotlib.pyplot as plt
from numpy import trapz  # Use trapz for integration

# Function for PINN
class NN(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i + 1]) for i in range(len(layers) - 1)])
        self.iter = 0
        self.Lbc = []
        self.Lpde = []
        self.Ltotal = []

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        x = (x - eta_min) / (eta_max - eta_min)
        for i in range(len(layers) - 2):
            z = self.linears[i](x)
            x = self.activation(z)
        return self.linears[-1](x)

    def bc_loss(self, x, y):
        x1, y1 = x[[0]], y[[0]]
        g1 = x1.clone().requires_grad_(True)
        f1 = self.forward(g1)
        f_etab1 = autograd.grad(f1, g1, torch.ones([x1.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        x2, y2 = x[[1]], y[[1]]
        g2 = x2.clone().requires_grad_(True)
        f2 = self.forward(g2)
        f_etab2 = autograd.grad(f2, g2, torch.ones([x2.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        loss_bc1 = self.loss_function(f1, y1) + self.loss_function(f_etab1, torch.zeros_like(f_etab1))
        loss_bc2 = self.loss_function(f_etab2, y2)
        loss_bc = loss_bc1 + loss_bc2
        self.Lbc.append(loss_bc.item())
        return loss_bc

    def PDE(self, x):
        g3 = x.clone().requires_grad_(True)
        f3 = self.forward(g3)
        f_eta = autograd.grad(f3, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta2 = autograd.grad(f_eta, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta3 = autograd.grad(f_eta2, g3, torch.ones([g3.shape[0], 1]), create_graph=True)[0]

        pde = f_eta3 + ((m + 1) / 2) * f3 * f_eta2 + m * (1 - (f_eta) ** 2)
        loss_pde = self.loss_function(pde, torch.zeros_like(pde))
        self.Lpde.append(loss_pde.item())
        return loss_pde

    def loss(self, x, xb, yb):
        loss1 = self.bc_loss(xb, yb)
        loss2 = self.PDE(x)
        total_loss = abs(loss1) + abs(loss2)
        self.Ltotal.append(total_loss.item())
        return total_loss

    def closure(self):
        optimizer.zero_grad()
        loss = self.loss(eta, bc, f_bc)
        loss.backward()
        self.iter += 1
        return loss    

# Function for PIELM
class PIELM(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.fc1 = nn.Linear(1, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1, bias=False)
        self.iter = 0
        self.Lbc = []
        self.Lpde = []
        self.Ltotal = []

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        x = (x - eta_min) / (eta_max - eta_min)
        z = self.fc1(x)
        a = self.activation(z)
        return self.fc2(a)

    def bc_loss(self, x, y):
        x1, y1 = x[[0]], y[[0]]
        g1 = x1.clone().requires_grad_(True)
        f1 = self.forward(g1)
        f_etab1 = autograd.grad(f1, g1, torch.ones([x1.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        x2, y2 = x[[1]], y[[1]]
        g2 = x2.clone().requires_grad_(True)
        f2 = self.forward(g2)
        f_etab2 = autograd.grad(f2, g2, torch.ones([x2.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        loss_bc1 = self.loss_function(f1, y1) + self.loss_function(f_etab1, torch.zeros_like(f_etab1))
        loss_bc2 = self.loss_function(f_etab2, y2)
        loss_bc = loss_bc1 + loss_bc2
        self.Lbc.append(loss_bc.item())
        return loss_bc

    def PDE(self, x):
        g3 = x.clone().requires_grad_(True)
        f3 = self.forward(g3)
        f_eta = autograd.grad(f3, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta2 = autograd.grad(f_eta, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta3 = autograd.grad(f_eta2, g3, torch.ones([g3.shape[0], 1]), create_graph=True)[0]
        pde = f_eta3 + ((m + 1) / 2) * f3 * f_eta2 + m * (1 - (f_eta) ** 2)
        loss_pde = self.loss_function(pde, torch.zeros_like(pde))
        self.Lpde.append(loss_pde.item())
        return loss_pde

    def loss(self, x, xb, yb):
        loss1 = self.bc_loss(xb, yb)
        loss2 = self.PDE(x)
        total_loss = abs(loss1) + abs(loss2)
        self.Ltotal.append(total_loss.item())  # Store total loss
        return total_loss

    def closure(self):
        optimizer.zero_grad()
        loss = self.loss(eta, bc, f_bc)
        loss.backward()
        self.iter += 1
        return loss

# Initialization and training for PINN
np.random.seed(941)
torch.manual_seed(11)
eta = np.linspace(0, 6, 1000)[:, None]
eta_min = eta[[0]]
eta_max = eta[[-1]]
m = 0
bc = eta[[0, -1]]
f_bc = np.array([[0], [1]])
eta = torch.from_numpy(eta).float()
eta_min = torch.from_numpy(eta_min).float()
eta_max = torch.from_numpy(eta_max).float()
bc = torch.from_numpy(bc).float()
f_bc = torch.from_numpy(f_bc).float()

layers = [1, 20, 1]
pinn = NN(layers)
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=0.1, max_iter=5000)
optimizer.step(pinn.closure)

# Initialization and training for PIELM
hidden_size = 50
pielm = PIELM()
optimizer = torch.optim.LBFGS(pielm.parameters(), lr=0.005, max_iter=20000)
optimizer.step(pielm.closure)

# Plot total loss vs iterations for comparison
plt.figure(figsize=(12, 6))
plt.plot(np.arange(1, len(pinn.Ltotal) + 1), np.log(pinn.Ltotal), marker='o', label='PINN Total Loss')
plt.plot(np.arange(1, len(pielm.Ltotal) + 1), np.log(pielm.Ltotal), marker='x', label='PIELM Total Loss')
plt.xlabel('Iteration')
plt.ylabel('ln(Total Loss)')
plt.title('Total Loss vs Iteration Comparison')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


#### For Stagnation point flow (m=1)

In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim
import numpy as np
import matplotlib.pyplot as plt
from numpy import trapz  # Use trapz for integration

# Function for PINN
class NN(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i + 1]) for i in range(len(layers) - 1)])
        self.iter = 0
        self.Lbc = []
        self.Lpde = []
        self.Ltotal = []

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        x = (x - eta_min) / (eta_max - eta_min)
        for i in range(len(layers) - 2):
            z = self.linears[i](x)
            x = self.activation(z)
        return self.linears[-1](x)

    def bc_loss(self, x, y):
        x1, y1 = x[[0]], y[[0]]
        g1 = x1.clone().requires_grad_(True)
        f1 = self.forward(g1)
        f_etab1 = autograd.grad(f1, g1, torch.ones([x1.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        x2, y2 = x[[1]], y[[1]]
        g2 = x2.clone().requires_grad_(True)
        f2 = self.forward(g2)
        f_etab2 = autograd.grad(f2, g2, torch.ones([x2.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        loss_bc1 = self.loss_function(f1, y1) + self.loss_function(f_etab1, torch.zeros_like(f_etab1))
        loss_bc2 = self.loss_function(f_etab2, y2)
        loss_bc = loss_bc1 + loss_bc2
        self.Lbc.append(loss_bc.item())
        return loss_bc

    def PDE(self, x):
        g3 = x.clone().requires_grad_(True)
        f3 = self.forward(g3)
        f_eta = autograd.grad(f3, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta2 = autograd.grad(f_eta, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta3 = autograd.grad(f_eta2, g3, torch.ones([g3.shape[0], 1]), create_graph=True)[0]

        pde = f_eta3 + ((m + 1) / 2) * f3 * f_eta2 + m * (1 - (f_eta) ** 2)
        loss_pde = self.loss_function(pde, torch.zeros_like(pde))
        self.Lpde.append(loss_pde.item())
        return loss_pde

    def loss(self, x, xb, yb):
        loss1 = self.bc_loss(xb, yb)
        loss2 = self.PDE(x)
        total_loss = abs(loss1) + abs(loss2)
        self.Ltotal.append(total_loss.item())
        return total_loss

    def closure(self):
        optimizer.zero_grad()
        loss = self.loss(eta, bc, f_bc)
        loss.backward()
        self.iter += 1
        return loss    

# Function for PIELM
class PIELM(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.fc1 = nn.Linear(1, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1, bias=False)
        self.iter = 0
        self.Lbc = []
        self.Lpde = []
        self.Ltotal = []

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        x = (x - eta_min) / (eta_max - eta_min)
        z = self.fc1(x)
        a = self.activation(z)
        return self.fc2(a)

    def bc_loss(self, x, y):
        x1, y1 = x[[0]], y[[0]]
        g1 = x1.clone().requires_grad_(True)
        f1 = self.forward(g1)
        f_etab1 = autograd.grad(f1, g1, torch.ones([x1.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        x2, y2 = x[[1]], y[[1]]
        g2 = x2.clone().requires_grad_(True)
        f2 = self.forward(g2)
        f_etab2 = autograd.grad(f2, g2, torch.ones([x2.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        loss_bc1 = self.loss_function(f1, y1) + self.loss_function(f_etab1, torch.zeros_like(f_etab1))
        loss_bc2 = self.loss_function(f_etab2, y2)
        loss_bc = loss_bc1 + loss_bc2
        self.Lbc.append(loss_bc.item())
        return loss_bc

    def PDE(self, x):
        g3 = x.clone().requires_grad_(True)
        f3 = self.forward(g3)
        f_eta = autograd.grad(f3, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta2 = autograd.grad(f_eta, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta3 = autograd.grad(f_eta2, g3, torch.ones([g3.shape[0], 1]), create_graph=True)[0]
        pde = f_eta3 + ((m + 1) / 2) * f3 * f_eta2 + m * (1 - (f_eta) ** 2)
        loss_pde = self.loss_function(pde, torch.zeros_like(pde))
        self.Lpde.append(loss_pde.item())
        return loss_pde

    def loss(self, x, xb, yb):
        loss1 = self.bc_loss(xb, yb)
        loss2 = self.PDE(x)
        total_loss = abs(loss1) + abs(loss2)
        self.Ltotal.append(total_loss.item())  # Store total loss
        return total_loss

    def closure(self):
        optimizer.zero_grad()
        loss = self.loss(eta, bc, f_bc)
        loss.backward()
        self.iter += 1
        return loss

# Initialization and training for PINN
np.random.seed(941)
torch.manual_seed(11)
eta = np.linspace(0, 6, 1000)[:, None]
eta_min = eta[[0]]
eta_max = eta[[-1]]
m = 1
bc = eta[[0, -1]]
f_bc = np.array([[0], [1]])
eta = torch.from_numpy(eta).float()
eta_min = torch.from_numpy(eta_min).float()
eta_max = torch.from_numpy(eta_max).float()
bc = torch.from_numpy(bc).float()
f_bc = torch.from_numpy(f_bc).float()

layers = [1, 20, 1]
pinn = NN(layers)
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=0.1, max_iter=5000)
optimizer.step(pinn.closure)

# Initialization and training for PIELM
hidden_size = 50
pielm = PIELM()
optimizer = torch.optim.LBFGS(pielm.parameters(), lr=0.005, max_iter=20000)
optimizer.step(pielm.closure)

# Plot total loss vs iterations for comparison
plt.figure(figsize=(12, 6))
plt.plot(np.arange(1, len(pinn.Ltotal) + 1), np.log(pinn.Ltotal), marker='o', label='PINN Total Loss')
plt.plot(np.arange(1, len(pielm.Ltotal) + 1), np.log(pielm.Ltotal), marker='x', label='PIELM Total Loss')
plt.xlabel('Iteration')
plt.ylabel('ln(Total Loss)')
plt.title('Total Loss vs Iteration Comparison')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


#### We can see that from the above comparison plot PIELM converges early as compared to PINNs but f' and f'' plots have slight inaccurate represntation through PIELM.

### 
## For Formulation A
######
#### For Flat plate (beta=0)

In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt

# First Implementation (PINN)
# Creation of 1D array (representing data sampling)
np.random.seed(941)
torch.manual_seed(11)
eta = np.linspace(0, 6, 1000)[:, None]
eta_min = eta[[0]]
eta_max = eta[[-1]]

# PDE parameters
beta = 0  # Set beta = 0 for flat plate case
bc = eta[[0, -1]]
f_bc = np.array([[0],[1]])

# Converting numpy arrays into tensors
eta = torch.from_numpy(eta).float()
eta_min = torch.from_numpy(eta_min).float()
eta_max = torch.from_numpy(eta_max).float()
bc = torch.from_numpy(bc).float()
f_bc = torch.from_numpy(f_bc).float()

class NN(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        self.Lbc = []
        self.Lpde = []
        self.Ltotal = []

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        x = (x - eta_min) / (eta_max - eta_min)
        for i in range(len(layers) - 2):
            z = self.linears[i](x)
            x = self.activation(z)
        a = self.linears[-1](x)
        return a

    def bc_loss(self, x, y):
        x1 = x[[0]]
        y1 = y[[0]]
        g1 = x1.clone()
        g1.requires_grad = True
        f1 = self.forward(g1)
        f_etab1 = autograd.grad(f1, g1, torch.ones([x1.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        x2 = x[[1]]
        y2 = y[[1]]
        g2 = x2.clone()
        g2.requires_grad = True
        f2 = self.forward(g2)
        f_etab2 = autograd.grad(f2, g2, torch.ones([x2.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        loss_bc1 = self.loss_function(f1, y1) + self.loss_function(f_etab1, torch.zeros_like(f_etab1))
        loss_bc2 = self.loss_function(f_etab2, y2)
        loss_bc = loss_bc1 + loss_bc2
        self.Lbc.append(loss_bc.item())
        return loss_bc
    
    def PDE(self, x):
        g3 = x.clone()
        g3.requires_grad = True
        f3 = self.forward(g3)
        f_eta = autograd.grad(f3, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta2 = autograd.grad(f_eta, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta3 = autograd.grad(f_eta2, g3, torch.ones([g3.shape[0], 1]), create_graph=True)[0]
        pde = f_eta3 + f3 * f_eta2 + beta * (1 - (f_eta)**2)
        loss_pde = self.loss_function(pde, torch.zeros_like(pde))
        self.Lpde.append(loss_pde.item())
        return loss_pde

    def loss(self, x, xb, yb):
        loss1 = self.bc_loss(xb, yb)
        loss2 = self.PDE(x)
        total_loss = abs(loss1) + abs(loss2)
        self.Ltotal.append(total_loss.item())
        return total_loss
    
    def closure(self):
        optimizer.zero_grad()
        loss = self.loss(eta, bc, f_bc)
        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0:
            print(f'Iteration {self.iter}: Loss = {loss.item()}')
        return loss    
    
# Specification of network architecture
layers = [1, 20, 1]
pinn = NN(layers)
max_iterations = 5000
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=0.1, max_iter=max_iterations,
                              tolerance_grad=1e-7, tolerance_change=1e-10,
                              history_size=500, line_search_fn='strong_wolfe')

optimizer.step(pinn.closure)
print(f'Total iterations: {pinn.iter}')

# Second Implementation (PIELM)
np.random.seed(42)
torch.manual_seed(42)
eta = np.linspace(0, 6, 1000)[:, None]
eta_min = eta[[0]]
eta_max = eta[[-1]]
beta = 0.0
bc = eta[[0, -1]]
f_bc = np.array([[0], [1]])

eta = torch.from_numpy(eta).float()
eta_min = torch.from_numpy(eta_min).float()
eta_max = torch.from_numpy(eta_max).float()
bc = torch.from_numpy(bc).float()
f_bc = torch.from_numpy(f_bc).float()

class PIELM(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.fc1 = nn.Linear(1, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1, bias=False)
        self.iter = 0
        self.Lbc = []
        self.Lpde = []

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        x = (x - eta_min) / (eta_max - eta_min)
        z = self.fc1(x)
        a = self.activation(z)
        f = self.fc2(a)
        return f

    def bc_loss(self, x, y):
        x1 = x[[0]]
        y1 = y[[0]]
        g1 = x1.clone()
        g1.requires_grad = True
        f1 = self.forward(g1)
        f_etab1 = autograd.grad(f1, g1, torch.ones([x1.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        x2 = x[[1]]
        y2 = y[[1]]
        g2 = x2.clone()
        g2.requires_grad = True
        f2 = self.forward(g2)
        f_etab2 = autograd.grad(f2, g2, torch.ones([x2.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        loss_bc1 = self.loss_function(f1, y1) + self.loss_function(f_etab1, torch.zeros_like(f_etab1))
        loss_bc2 = self.loss_function(f_etab2, y2)
        loss_bc = loss_bc1 + loss_bc2
        self.Lbc.append(loss_bc.item())
        return loss_bc

    def PDE(self, x):
        g3 = x.clone()
        g3.requires_grad = True
        f3 = self.forward(g3)
        f_eta = autograd.grad(f3, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta2 = autograd.grad(f_eta, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta3 = autograd.grad(f_eta2, g3, torch.ones([g3.shape[0], 1]), create_graph=True)[0]
        pde = f_eta3 + f3 * f_eta2 + beta * (1 - f_eta**2)
        loss_pde = self.loss_function(pde, torch.zeros_like(pde))
        self.Lpde.append(loss_pde.item())
        return loss_pde

    def loss(self, x, xb, yb):
        loss1 = self.bc_loss(xb, yb)
        loss2 = self.PDE(x)
        total_loss = loss1 + loss2  # Total loss for PIELM
        return total_loss

    def closure(self):
        optimizer.zero_grad()
        loss = self.loss(eta, bc, f_bc)
        loss.backward()
        return loss    

# Hidden size for PIELM
hidden_size = 20
pielm = PIELM()
optimizer = torch.optim.LBFGS(pielm.parameters(), lr=0.1, max_iter=5000,
                               tolerance_grad=1e-7, tolerance_change=1e-10)

optimizer.step(pielm.closure)

# Comparison of Total Losses
plt.figure(figsize=(10, 6))
plt.plot(pinn.Ltotal, label='PINN Total Loss',marker='o', color='blue')
plt.plot(pielm.Lbc, label='PIELM Total Loss', marker='x', color='orange')  # Change to total loss for PIELM
plt.yscale('log')
plt.title('Comparison of Total Losses')
plt.xlabel('Iteration')
plt.ylabel('Loss (Log Scale)')
plt.legend()
plt.grid()
plt.show()


#### For Stagnation point flow (beta=1)

In [ ]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt

# First Implementation (PINN)
# Creation of 1D array (representing data sampling)
np.random.seed(941)
torch.manual_seed(11)
eta = np.linspace(0, 6, 1000)[:, None]
eta_min = eta[[0]]
eta_max = eta[[-1]]

# PDE parameters
beta = 0  # Set beta = 0 for flat plate case
bc = eta[[0, -1]]
f_bc = np.array([[0],[1]])

# Converting numpy arrays into tensors
eta = torch.from_numpy(eta).float()
eta_min = torch.from_numpy(eta_min).float()
eta_max = torch.from_numpy(eta_max).float()
bc = torch.from_numpy(bc).float()
f_bc = torch.from_numpy(f_bc).float()

class NN(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        self.Lbc = []
        self.Lpde = []
        self.Ltotal = []

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        x = (x - eta_min) / (eta_max - eta_min)
        for i in range(len(layers) - 2):
            z = self.linears[i](x)
            x = self.activation(z)
        a = self.linears[-1](x)
        return a

    def bc_loss(self, x, y):
        x1 = x[[0]]
        y1 = y[[0]]
        g1 = x1.clone()
        g1.requires_grad = True
        f1 = self.forward(g1)
        f_etab1 = autograd.grad(f1, g1, torch.ones([x1.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        x2 = x[[1]]
        y2 = y[[1]]
        g2 = x2.clone()
        g2.requires_grad = True
        f2 = self.forward(g2)
        f_etab2 = autograd.grad(f2, g2, torch.ones([x2.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        loss_bc1 = self.loss_function(f1, y1) + self.loss_function(f_etab1, torch.zeros_like(f_etab1))
        loss_bc2 = self.loss_function(f_etab2, y2)
        loss_bc = loss_bc1 + loss_bc2
        self.Lbc.append(loss_bc.item())
        return loss_bc
    
    def PDE(self, x):
        g3 = x.clone()
        g3.requires_grad = True
        f3 = self.forward(g3)
        f_eta = autograd.grad(f3, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta2 = autograd.grad(f_eta, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta3 = autograd.grad(f_eta2, g3, torch.ones([g3.shape[0], 1]), create_graph=True)[0]
        pde = f_eta3 + f3 * f_eta2 + beta * (1 - (f_eta)**2)
        loss_pde = self.loss_function(pde, torch.zeros_like(pde))
        self.Lpde.append(loss_pde.item())
        return loss_pde

    def loss(self, x, xb, yb):
        loss1 = self.bc_loss(xb, yb)
        loss2 = self.PDE(x)
        total_loss = abs(loss1) + abs(loss2)
        self.Ltotal.append(total_loss.item())
        return total_loss
    
    def closure(self):
        optimizer.zero_grad()
        loss = self.loss(eta, bc, f_bc)
        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0:
            print(f'Iteration {self.iter}: Loss = {loss.item()}')
        return loss    
    
# Specification of network architecture
layers = [1, 20, 1]
pinn = NN(layers)
max_iterations = 5000
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=0.1, max_iter=max_iterations,
                              tolerance_grad=1e-7, tolerance_change=1e-10,
                              history_size=500, line_search_fn='strong_wolfe')

optimizer.step(pinn.closure)
print(f'Total iterations: {pinn.iter}')

# Second Implementation (PIELM)
np.random.seed(42)
torch.manual_seed(42)
eta = np.linspace(0, 6, 1000)[:, None]
eta_min = eta[[0]]
eta_max = eta[[-1]]
beta = 1
bc = eta[[0, -1]]
f_bc = np.array([[0], [1]])

eta = torch.from_numpy(eta).float()
eta_min = torch.from_numpy(eta_min).float()
eta_max = torch.from_numpy(eta_max).float()
bc = torch.from_numpy(bc).float()
f_bc = torch.from_numpy(f_bc).float()

class PIELM(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction='mean')
        self.fc1 = nn.Linear(1, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1, bias=False)
        self.iter = 0
        self.Lbc = []
        self.Lpde = []

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x)
        x = (x - eta_min) / (eta_max - eta_min)
        z = self.fc1(x)
        a = self.activation(z)
        f = self.fc2(a)
        return f

    def bc_loss(self, x, y):
        x1 = x[[0]]
        y1 = y[[0]]
        g1 = x1.clone()
        g1.requires_grad = True
        f1 = self.forward(g1)
        f_etab1 = autograd.grad(f1, g1, torch.ones([x1.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        x2 = x[[1]]
        y2 = y[[1]]
        g2 = x2.clone()
        g2.requires_grad = True
        f2 = self.forward(g2)
        f_etab2 = autograd.grad(f2, g2, torch.ones([x2.shape[0], 1]), retain_graph=True, create_graph=True)[0]

        loss_bc1 = self.loss_function(f1, y1) + self.loss_function(f_etab1, torch.zeros_like(f_etab1))
        loss_bc2 = self.loss_function(f_etab2, y2)
        loss_bc = loss_bc1 + loss_bc2
        self.Lbc.append(loss_bc.item())
        return loss_bc

    def PDE(self, x):
        g3 = x.clone()
        g3.requires_grad = True
        f3 = self.forward(g3)
        f_eta = autograd.grad(f3, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta2 = autograd.grad(f_eta, g3, torch.ones([g3.shape[0], 1]), retain_graph=True, create_graph=True)[0]
        f_eta3 = autograd.grad(f_eta2, g3, torch.ones([g3.shape[0], 1]), create_graph=True)[0]
        pde = f_eta3 + f3 * f_eta2 + beta * (1 - f_eta**2)
        loss_pde = self.loss_function(pde, torch.zeros_like(pde))
        self.Lpde.append(loss_pde.item())
        return loss_pde

    def loss(self, x, xb, yb):
        loss1 = self.bc_loss(xb, yb)
        loss2 = self.PDE(x)
        total_loss = loss1 + loss2  # Total loss for PIELM
        return total_loss

    def closure(self):
        optimizer.zero_grad()
        loss = self.loss(eta, bc, f_bc)
        loss.backward()
        return loss    

# Hidden size for PIELM
hidden_size = 20
pielm = PIELM()
optimizer = torch.optim.LBFGS(pielm.parameters(), lr=0.1, max_iter=5000,
                               tolerance_grad=1e-7, tolerance_change=1e-10)

optimizer.step(pielm.closure)

# Comparison of Total Losses
plt.figure(figsize=(10, 6))
plt.plot(pinn.Ltotal, label='PINN Total Loss',marker='o', color='blue')
plt.plot(pielm.Lbc, label='PIELM Total Loss', marker='x', color='orange')  # Change to total loss for PIELM
plt.yscale('log')
plt.title('Comparison of Total Losses')
plt.xlabel('Iteration')
plt.ylabel('Loss (Log Scale)')
plt.legend()
plt.grid()
plt.show()
